# Load Data with Pandas
* Reference to paraatm@https://github.com/ymlasu/para-atm/tree/master/paraatm

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pkg_resources.extern import packaging

def parse_version(v):
    try:
        return packaging.version.Version(v)
    except packaging.version.InvalidVersion:
        return packaging.version.LegacyVersion(v)

In [3]:
def read_iff_file(filename, record_types=3, callsigns=None, chunksize=50000, encoding='latin-1'):
    """
    Read IFF file and return data frames for requested record types
    
    From IFF 2.15 specification, record types include:
    2. header
    3. track point
    4. flight plan
    5. data source program
    6. sectorization
    7. minimum safe altitude
    8. flight progress
    9. aircraft state
    Parameters
    ----------
    filename : str
        File to read
    record_types : int, sequence of ints, or 'all'
        Record types to return
    callsigns : None, string, or list of strings
        If None, return records for all aircraft callsigns.
        Otherwise, only return records that match the given callsign
        (in the case of a single string) or match one of the specified
        callsigns (in the case of a list of strings).
    chunksize: int
        Number of rows that are read at a time by pd.read_csv.  This
        limits memory usage when working with large files, as we can
        extract out the desired rows from each chunk, intead of
        reading everything into one large DataFrame and then taking a
        subset.
    encoding: str
        Encoding argument passed on to open and pd.read_csv.  Using
        'latin-1' instead of the default will suppress errors that
        might otherwise occur with minor data corruption.  See
        http://python-notes.curiousefficiency.org/en/latest/python3/text_file_processing.html
    
    Returns
    -------
    DataFrame or dict of DataFrames
       If record_types is a scalar, return a DataFrame containing the
       data for that record type only.  Otherwise, return a dictionary
       mapping each requested record type to a corresponding DataFrame.
    """
    # Note default record_type of 3 (track point) is used for
    # consistency with the behavior of other functions that expect
    # flight tracking data

    # Determine file format version.  This is in record type 1, which
    # for now we assume to occur on the first line.
    with open(filename, 'r') as f:
        version = parse_version(f.readline().split(',')[2])

    # Columns for each record type, from version 2.6 specification.
    cols = {0:['recType','comment'],
            1:['recType','fileType','fileFormatVersion'],
            2:['recType','recTime','fltKey','bcnCode','cid','Source','msgType','AcId','recTypeCat','acType','Orig','Dest','opsType','estOrig','estDest'],
            3:['recType','recTime','fltKey','bcnCode','cid','Source','msgType','AcId','recTypeCat','coord1','coord2','alt','significance','coord1Accur','coord2Accur','altAccur','groundSpeed','course','rateOfClimb','altQualifier','altIndicator','trackPtStatus','leaderDir','scratchPad','msawInhibitInd','assignedAltString','controllingFac','controllingSeg','receivingFac','receivingSec','activeContr','primaryContr','kybrdSubset','kybrdSymbol','adsCode','opsType','airportCode'],
            4:['recType','recTime','fltKey','bcnCode','cid','Source','msgType','AcId','recTypeCat','acType','Orig','Dest','altcode','alt','maxAlt','assignedAltString','requestedAltString','route','estTime','fltCat','perfCat','opsType','equipList','coordinationTime','coordinationTimeType','leaderDir','scratchPad1','scratchPad2','fixPairScratchPad','prefDepArrRoute','prefDepRoute','prefArrRoute'],
            5:['recType','dataSource','programName','programVersion'],
            6:['recType','recTime','Source','msgType','rectypeCat','sectorizationString'],
            7:['recType','recTime','fltKey','bcnCode','cid','Source','msgType','AcId','recTypeCat','coord1','coord2','alt','significance','coord1Accur','coord2Accur','altAccur','msawtype','msawTimeCat','msawLocCat','msawMinSafeAlt','msawIndex1','msawIndex2','msawVolID'],
            8:['recType','recTime','fltKey','bcnCode','cid','Source','msgType','AcId','recTypeCat','acType','Orig','Dest','depTime','depTimeType','arrTime','arrTimeType'],
            9:['recType','recTime','fltKey','bcnCode','cid','Source','msgType','AcId','recTypeCat','coord1','coord2','alt','pitchAngle','trueHeading','rollAngle','trueAirSpeed','fltPhaseIndicator'],
            10:['recType','recTime','fltKey','bcnCode','cid','Source','msgType','AcId','recTypeCat','configType','configSpec']}

    # For newer versions, additional columns are supported.  However,
    # this code could be commented out, and it should still be
    # compatible with newer versions, but just ignoring the additional
    # columns.
    if version >= parse_version('2.13'):
        cols[2] += ['modeSCode']
        cols[3] += ['trackNumber','tptReturnType','modeSCode']
        cols[4] += ['coordinationPoint','coordinationPointType','trackNumber','modeSCode']
    if version >= parse_version('2.15'):
        cols[3] += ['sensorTrackNumberList','spi','dvs','dupM3a','tid']

    # Determine the record type of each row
    with open(filename, 'r', encoding=encoding) as f:
        # An alternative, using less memory, would be to directly
        # create skiprows indices for a particular record type, using
        # a comprehension on enumerate(f); however, that would not
        # allow handling multiple record types.
        line_record_types = [int(line.split(',')[0]) for line in f]

    # Determine which record types to retrieve, and whether the result
    # should be a scalar or dict:
    if record_types == 'all':
        record_types = np.unique(line_record_types)
        scalar_result = False
    elif hasattr(record_types, '__getitem__'):
        scalar_result = False
    else:
        record_types = [record_types]
        scalar_result = True

    if callsigns is not None:
        callsigns = list(np.atleast_1d(callsigns))


    data_frames = dict()
    for record_type in record_types:
        # Construct list of rows to skip:
        skiprows = [i for i,lr in enumerate(line_record_types) if lr != record_type]
        
        # Passing usecols is necessary because for some records, the
        # actual data has extraneous empty columns at the end, in which
        # case the data does not seem to get read correctly without
        # usecols
        if callsigns is None:
            df = pd.concat((chunk for chunk in pd.read_csv(filename, header=None, skiprows=skiprows, names=cols[record_type], usecols=cols[record_type], na_values='?', encoding=encoding, chunksize=chunksize, low_memory=False)), ignore_index=True)
        else:
            df = pd.concat((chunk[chunk['AcId'].isin(callsigns)] for chunk in pd.read_csv(filename, header=None, skiprows=skiprows, names=cols[record_type], usecols=cols[record_type], na_values='?', encoding=encoding, chunksize=chunksize, low_memory=False)), ignore_index=True)

        # For consistency with other PARA-ATM data:
        df.rename(columns={'recTime':'time',
                           'AcId':'callsign',
                           'coord1':'latitude',
                           'coord2':'longitude',
                           'alt':'altitude',
                           'rateOfClimb':'rocd',
                           'groundSpeed':'tas',
                           'course':'heading'},
                  inplace=True)

        if 'time' in df:
            df['time'] = pd.to_datetime(df['time'], unit='s')
        if 'altitude' in df:
            df['altitude'] *= 100 # Convert 100s ft to ft

        # Store to dict of data frames
        data_frames[record_type] = df

    if scalar_result:
        result = data_frames[record_types[0]]
    else:
        result = data_frames

    return result

# Specify Date for Sector IFF

In [4]:
date = 20190624
import glob
file_path = glob.glob("/home/pzhao28/Documents/Research/Software/data/ZTL/IFF_ZTL_{}*.csv".format(date))[0]

# Load into Pandas

In [5]:
pd_df = read_iff_file(file_path, record_types=2, chunksize = 1e6)

In [7]:
pd_df

,recType,time,fltKey,bcnCode,cid,Source,msgType,callsign,recTypeCat,acType,Orig,Dest,opsType,estOrig,estDest,modeSCode
0,2,2019-06-24 04:33:12,159502,2572.0,592,0/ZTL,NaN,DAL2881,1,B712,KATL,KCID,D,NaN,NaN,AD8B25
1,2,2019-06-24 04:38:03,159503,2010.0,474,0/ZTL,NaN,DAL1974,1,A321,KATL,KTPA,D,NaN,NaN,A3E8DC
2,2,2019-06-24 04:01:58,159504,4073.0,230,0/ZTL,NaN,DAL2706,1,B738,SINCA352034,KMCO,O,NaN,NaN,A4463E
3,2,2019-06-24 04:11:31,159505,713.0,975,0/ZTL,NaN,GTI604,1,B748,VXV339038,KHSV,O,NaN,NaN,ABA800
4,2,2019-06-24 04:08:16,159507,2576.0,125,0/ZTL,NaN,SWA4038,1,B737,VUZ106021,KSTL,O,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9566,2,2019-06-25 04:19:14,177125,2460.0,029,0/ZTL,NaN,UAL285,1,B737,CAE019048,KDCA,O,NaN,NaN,A10E8B
9567,2,2019-06-25 04:30:03,177203,NaN,918,0/ZTL,NaN,PDT4720,1,E145,KCLT,KCHO,D,NaN,NaN,A88CBF
9568,2,2019-06-25 04:41:39,177302,2417.0,443,0/ZTL,NaN,FFT1638,1,A320,GSO193050,KRDU,O,NaN,NaN,NaN
9569,2,2019-06-25 04:27:17,177304,3157.0,529,0/ZTL,NaN,DAL2502,1,MD90,GSO195045,KRDU,D,NaN,NaN,ACDFF1


# Convert Pandas into PySpark df

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [9]:
spark = SparkSession \
        .builder \
        .appName("Terminal_Area_Flight_Data_Query") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [10]:
df = spark.createDataFrame(pd_df.astype(str))

In [11]:
df.show(5)

+-------+--------------------+------+-------+---+------+-------+--------+----------+------+-----------+----+-------+-------+-------+---------+
|recType|                time|fltKey|bcnCode|cid|Source|msgType|callsign|recTypeCat|acType|       Orig|Dest|opsType|estOrig|estDest|modeSCode|
+-------+--------------------+------+-------+---+------+-------+--------+----------+------+-----------+----+-------+-------+-------+---------+
|      2|2019-06-24 04:33:...|159502| 2572.0|592| 0/ZTL|    nan| DAL2881|         1|  B712|       KATL|KCID|      D|    nan|    nan|   AD8B25|
|      2|2019-06-24 04:38:...|159503| 2010.0|474| 0/ZTL|    nan| DAL1974|         1|  A321|       KATL|KTPA|      D|    nan|    nan|   A3E8DC|
|      2|2019-06-24 04:01:...|159504| 4073.0|230| 0/ZTL|    nan| DAL2706|         1|  B738|SINCA352034|KMCO|      O|    nan|    nan|   A4463E|
|      2|2019-06-24 04:11:...|159505|  713.0|975| 0/ZTL|    nan|  GTI604|         1|  B748|  VXV339038|KHSV|      O|    nan|    nan|   ABA800|

In [39]:
df.printSchema()

root
 |-- recType: string (nullable = true)
 |-- time: string (nullable = true)
 |-- fltKey: string (nullable = true)
 |-- bcnCode: string (nullable = true)
 |-- cid: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- msgType: string (nullable = true)
 |-- callsign: string (nullable = true)
 |-- recTypeCat: string (nullable = true)
 |-- acType: string (nullable = true)
 |-- Orig: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- opsType: string (nullable = true)
 |-- estOrig: string (nullable = true)
 |-- estDest: string (nullable = true)
 |-- modeSCode: string (nullable = true)

